In [67]:
import pyranges as pr
import pandas as pd

In [10]:
gtf = 'tests/files/Canx.gtf'
dist = 50

gr_gtf = pr.read_gtf(gtf)

df_tss = gr_gtf.features.tss()#.extend(dist)

In [11]:
df_tss.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,transcript_type,transcript_status,transcript_name,protein_id,tag,transcript_support_level,ccdsid,exon_number,exon_id,havana_transcript
0,chr11,ENSEMBL,tss,50325672,50325673,.,-,.,ENSMUSG00000020368.14,protein_coding,...,protein_coding,KNOWN,Canx-201,ENSMUSP00000137440.1,CCDS,1,CCDS24633.1,NaN,NaN,NaN
1,chr11,HAVANA,tss,50325672,50325673,.,-,.,ENSMUSG00000020368.14,protein_coding,...,protein_coding,KNOWN,Canx-001,ENSMUSP00000020637.8,CCDS,1,CCDS24633.1,NaN,NaN,OTTMUST00000012410.1
2,chr11,HAVANA,tss,50299377,50299378,.,-,.,ENSMUSG00000020368.14,protein_coding,...,processed_transcript,KNOWN,Canx-003,NaN,NaN,2,NaN,NaN,NaN,OTTMUST00000012412.1
3,chr11,HAVANA,tss,50298318,50298319,.,-,.,ENSMUSG00000020368.14,protein_coding,...,processed_transcript,KNOWN,Canx-002,NaN,NaN,5,NaN,NaN,NaN,OTTMUST00000012411.1
4,chr11,HAVANA,tss,50300989,50300990,.,-,.,ENSMUSG00000020368.14,protein_coding,...,processed_transcript,KNOWN,Canx-004,NaN,NaN,3,NaN,NaN,NaN,OTTMUST00000012413.1


In [ ]:
d = {'Chromosome': ['chr1', 'chr1', 'chr1'], 'Start': [3, 8, 5], 'End': [6, 9, 7],
     'Strand': ['+', '+', '-']}
gr = pr.from_dict(d)
gr

In [15]:
tss_bed = 'tests/files/Canx_tss.bed'
tes_bed = 'tests/files/Canx_tes.bed'
gtf = 'tests/files/Canx.gtf'

In [16]:
gtf = pr.read_gtf(gtf, duplicate_attr=True)

In [17]:
gtf.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,transcript_type,transcript_status,transcript_name,protein_id,tag,transcript_support_level,ccdsid,exon_number,exon_id,havana_transcript
0,chr11,HAVANA,gene,50293960,50325673,.,-,.,ENSMUSG00000020368.14,protein_coding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr11,ENSEMBL,transcript,50293960,50325673,.,-,.,ENSMUSG00000020368.14,protein_coding,...,protein_coding,KNOWN,Canx-201,ENSMUSP00000137440.1,"basic,appris_principal_1,CCDS",1,CCDS24633.1,NaN,NaN,NaN
2,chr11,ENSEMBL,exon,50325492,50325673,.,-,.,ENSMUSG00000020368.14,protein_coding,...,protein_coding,KNOWN,Canx-201,ENSMUSP00000137440.1,"basic,appris_principal_1,CCDS",1,CCDS24633.1,1,ENSMUSE00000371309.6,NaN
3,chr11,ENSEMBL,exon,50311603,50311780,.,-,.,ENSMUSG00000020368.14,protein_coding,...,protein_coding,KNOWN,Canx-201,ENSMUSP00000137440.1,"basic,appris_principal_1,CCDS",1,CCDS24633.1,2,ENSMUSE00001016144.1,NaN
4,chr11,ENSEMBL,CDS,50311603,50311777,.,-,0,ENSMUSG00000020368.14,protein_coding,...,protein_coding,KNOWN,Canx-201,ENSMUSP00000137440.1,"basic,appris_principal_1,CCDS",1,CCDS24633.1,2,ENSMUSE00001016144.1,NaN
5,chr11,ENSEMBL,start_codon,50311774,50311777,.,-,0,ENSMUSG00000020368.14,protein_coding,...,protein_coding,KNOWN,Canx-201,ENSMUSP00000137440.1,"basic,appris_principal_1,CCDS",1,CCDS24633.1,2,ENSMUSE00001016144.1,NaN
6,chr11,ENSEMBL,exon,50310793,50310867,.,-,.,ENSMUSG00000020368.14,protein_coding,...,protein_coding,KNOWN,Canx-201,ENSMUSP00000137440.1,"basic,appris_principal_1,CCDS",1,CCDS24633.1,3,ENSMUSE00000103860.1,NaN
7,chr11,ENSEMBL,CDS,50310793,50310867,.,-,0,ENSMUSG00000020368.14,protein_coding,...,protein_coding,KNOWN,Canx-201,ENSMUSP00000137440.1,"basic,appris_principal_1,CCDS",1,CCDS24633.1,3,ENSMUSE00000103860.1,NaN


In [18]:
gtf = gtf.df

In [19]:
gtf = gtf.loc[gtf.Feature == 'transcript']


In [20]:
gtf.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,transcript_type,transcript_status,transcript_name,protein_id,tag,transcript_support_level,ccdsid,exon_number,exon_id,havana_transcript
1,chr11,ENSEMBL,transcript,50293960,50325673,.,-,.,ENSMUSG00000020368.14,protein_coding,...,protein_coding,KNOWN,Canx-201,ENSMUSP00000137440.1,"basic,appris_principal_1,CCDS",1,CCDS24633.1,NaN,NaN,NaN
36,chr11,HAVANA,transcript,50294466,50325673,.,-,.,ENSMUSG00000020368.14,protein_coding,...,protein_coding,KNOWN,Canx-001,ENSMUSP00000020637.8,"basic,appris_principal_1,CCDS",1,CCDS24633.1,NaN,NaN,OTTMUST00000012410.1
71,chr11,HAVANA,transcript,50296198,50299378,.,-,.,ENSMUSG00000020368.14,protein_coding,...,processed_transcript,KNOWN,Canx-003,NaN,NaN,2,NaN,NaN,NaN,OTTMUST00000012412.1
77,chr11,HAVANA,transcript,50297363,50298319,.,-,.,ENSMUSG00000020368.14,protein_coding,...,processed_transcript,KNOWN,Canx-002,NaN,NaN,5,NaN,NaN,NaN,OTTMUST00000012411.1
80,chr11,HAVANA,transcript,50298837,50300990,.,-,.,ENSMUSG00000020368.14,protein_coding,...,processed_transcript,KNOWN,Canx-004,NaN,NaN,3,NaN,NaN,NaN,OTTMUST00000012413.1


In [21]:
gtf['basic_set'] = gtf.tag.str.contains('basic')

In [53]:
gtf['temp'] = gtf.tag.str.split('appris_principal_', n=1, expand=True)[1]
gtf['appris_principal'] = gtf.temp.str.split(',', n=1, expand=True)[0]

In [54]:
gtf.basic_set.fillna(False, inplace=True)
gtf = gtf.sort_values(by=['appris_principal', 'basic_set'], 
                      ascending=[False, True],
                      na_position='last')

In [55]:

gtf

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,protein_id,tag,transcript_support_level,ccdsid,exon_number,exon_id,havana_transcript,basic_set,temp,appris_principal
1,chr11,ENSEMBL,transcript,50293960,50325673,.,-,.,ENSMUSG00000020368.14,protein_coding,...,ENSMUSP00000137440.1,"basic,appris_principal_1,CCDS",1,CCDS24633.1,NaN,NaN,NaN,True,"1,CCDS",1
36,chr11,HAVANA,transcript,50294466,50325673,.,-,.,ENSMUSG00000020368.14,protein_coding,...,ENSMUSP00000020637.8,"basic,appris_principal_1,CCDS",1,CCDS24633.1,NaN,NaN,OTTMUST00000012410.1,True,"1,CCDS",1
71,chr11,HAVANA,transcript,50296198,50299378,.,-,.,ENSMUSG00000020368.14,protein_coding,...,NaN,NaN,2,NaN,NaN,NaN,OTTMUST00000012412.1,False,NaN,NaN
77,chr11,HAVANA,transcript,50297363,50298319,.,-,.,ENSMUSG00000020368.14,protein_coding,...,NaN,NaN,5,NaN,NaN,NaN,OTTMUST00000012411.1,False,NaN,NaN
80,chr11,HAVANA,transcript,50298837,50300990,.,-,.,ENSMUSG00000020368.14,protein_coding,...,NaN,NaN,3,NaN,NaN,NaN,OTTMUST00000012413.1,False,NaN,NaN


## compute triplets

In [310]:
def get_transcript_ref(fname):
    
    df = pr.read_gtf(fname, duplicate_attr=True).df
    df = df.loc[df.Feature == 'transcript']
    df['basic_set'] = df.tag.str.contains('basic')
    df['temp'] = df.tag.str.split('appris_principal_', n=1, expand=True)[1]
    df['appris_principal'] = df.temp.str.split(',', n=1, expand=True)[0]
    
    return df

In [311]:
def number_tss_ic_tes(df, mode):
    # groupby feature but record which feature
    # each transcript id uses 
    cols = ['transcript_id', mode, 'basic_set', 'appris_principal', 'gene_id']
    df = df[cols].groupby([mode, 'gene_id']).agg({'transcript_id': ','.join,
                                     'basic_set': 'max',
                                     'appris_principal': 'min'}).reset_index()

    # compute intron chain number
    df['{}_num'.format(mode)] = df.sort_values(by=['gene_id', 'appris_principal', 'basic_set'],
                                 ascending=[True, False, True],
                                 na_position='last')\
                                 .groupby(['gene_id'])\
                                 .cumcount() + 1
    
    return df

In [312]:
gtf = 'tests/files/Canx.gtf'
tss_bed = 'tests/files/Canx_tss.bed'
tes_bed = 'tests/files/Canx_tes.bed'

In [351]:
### intron chain annotation ###

# get basic status and appris_principal tag for each transcript
t_df = get_transcript_ref(fname)
t_df = t_df[['transcript_id', 'gene_id', 'basic_set', 'appris_principal']]

# get unique intron chains from gtf
df = pr.read_gtf(fname).df

# restrict to exon entries
df = df.loc[df.Feature == 'exon']
cols = ['Chromosome', 'Strand', 'Start', 'End', 'transcript_id', 'gene_id']
df = df[cols]

# melt to isolate individual coordinates
df = pd.melt(df, id_vars=['Chromosome', 'Strand', 'transcript_id', 'gene_id'],
            value_vars=['Start', 'End'],
            value_name='Coord')
df.drop('variable', axis=1, inplace=True)

# sort to order coordinates correctly
df.Coord = df.Coord.astype(int)
fwd = df.loc[df.Strand == '+'].copy(deep=True)
rev = df.loc[df.Strand == '-'].copy(deep=True)

fwd.sort_values(by=['Chromosome', 'transcript_id', 'Coord'],
                ascending=[True, True, True])
rev.sort_values(by=['Chromosome', 'transcript_id', 'Coord'],
                ascending=[True, True, False])
df = pd.concat([fwd, rev])

# create intron chain strings
df.Coord = df.Coord.astype(str)
df = df.groupby(['Chromosome', 'Strand', 'transcript_id'])['Coord'].apply('-'.join).reset_index()

# remove tss and tes from intron chain
df['temp'] = df.Coord.str.split('-', n=1, expand=True)[1]
df['ic'] = df.temp.str.rsplit('-', n=1, expand=True)[0]

# add basic annotation, appris principal number, and gene id
df = df.merge(t_df, on='transcript_id', how='left')

# add number for each unique intron chain
df = number_tss_ic_tes(df, mode='ic')

# keep track of which transcripts use each intron chain
df.transcript_id = df.transcript_id.str.split(',')
ttrip_df = df[['transcript_id', 'ic_num']].explode(column='transcript_id')

In [352]:
### tss / tes numbering ###
beds = {}
for mode, bed_file in zip(['tss', 'tes'], [tss_bed, tes_bed]):

    ends = pr.read_bed(bed_file).df
    ends.rename({'ThickStart': 'gene_id'}, axis=1, inplace=True)
    ends[mode] = [i for i in range(len(ends.index))]
    ends = pr.PyRanges(ends)

    # merge gtf ends with bed ends
    df = pr.read_gtf(gtf)
    if mode == 'tss':
        df = df.features.tss().df
    elif mode == 'tes':
        df = df.features.tes().df

    # merge in information about transcript status
    t_df = get_transcript_ref(gtf)
    t_df = t_df[['transcript_id', 'appris_principal', 'basic_set']]
    df = df.merge(t_df, how='left', on='transcript_id')
    df = pr.PyRanges(df)

    # join with bed ends
    df = df.join(ends, 
                 strandedness='same',
                 how='left').df
    df = df.loc[df.gene_id == df.gene_id_b]

    # add number for each unique end region
    df = number_tss_ic_tes(df, mode=mode)
    
    # keep track of which transcripts use each intron chain
    df.transcript_id = df.transcript_id.str.split(',')
    temp = df[['transcript_id', '{}_num'.format(mode)]].explode(column='transcript_id')  
    ttrip_df = ttrip_df.merge(temp, on='transcript_id')   
    
    # add numbers back into bed file
    ends = ends.df
    ends = ends.merge(df[[mode, '{}_num'.format(mode)]], on=mode)
    
    ends['Name'] = ends.gene_id+'_'+ends['{}_num'.format(mode)].astype(str)
    ends.drop([mode, 'gene_id', '{}_num'.format(mode)], axis=1, inplace=True)
    beds[mode] = pr.PyRanges(ends)

In [353]:
# create formatted string to represent each transcript
ttrip_df['transcript_triplet'] = '['+ttrip_df['tss_num'].astype(str)+', '+\
                                     ttrip_df['ic_num'].astype(str)+', '+\
                                     ttrip_df['tes_num'].astype(str)+']'

In [354]:
ttrip_df

,transcript_id,ic_num,tss_num,tes_num,transcript_triplet
0,ENSMUST00000153068.1,4,4,5,"[4, 4, 5]"
1,ENSMUST00000146979.1,5,3,4,"[3, 5, 4]"
2,ENSMUST00000155801.1,1,2,3,"[2, 1, 3]"
3,ENSMUST00000155801.2,1,2,3,"[2, 1, 3]"
4,ENSMUST00000179865.6,2,1,1,"[1, 2, 1]"
5,ENSMUST00000020637.8,3,1,2,"[1, 3, 2]"


In [355]:
### output files ###

# gtf
df = pr.read_gtf(gtf, duplicate_attr=True).df
df = df.merge(ttrip_df[['transcript_id', 'transcript_triplet']], 
              how='left',
              on='transcript_id')
df = pr.PyRanges(df)

In [356]:
beds['tss']

,Chromosome,Start,End,Name,Score,Strand
0,chr11,50325622,50325723,ENSMUSG00000020368.14_1,.,-
1,chr11,50299327,50299428,ENSMUSG00000020368.14_3,.,-
2,chr11,50298268,50298369,ENSMUSG00000020368.14_4,.,-
3,chr11,50300939,50301040,ENSMUSG00000020368.14_2,.,-


In [338]:
df.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,transcript_status,transcript_name,protein_id,tag,transcript_support_level,ccdsid,exon_number,exon_id,havana_transcript,transcript_triplet
0,chr11,HAVANA,gene,50293960,50325673,.,-,.,ENSMUSG00000020368.14,protein_coding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr11,ENSEMBL,transcript,50293960,50325673,.,-,.,ENSMUSG00000020368.14,protein_coding,...,KNOWN,Canx-201,ENSMUSP00000137440.1,"basic,appris_principal_1,CCDS",1,CCDS24633.1,NaN,NaN,NaN,"[1, 2, 1]"
2,chr11,ENSEMBL,exon,50325492,50325673,.,-,.,ENSMUSG00000020368.14,protein_coding,...,KNOWN,Canx-201,ENSMUSP00000137440.1,"basic,appris_principal_1,CCDS",1,CCDS24633.1,1,ENSMUSE00000371309.6,NaN,"[1, 2, 1]"
3,chr11,ENSEMBL,exon,50311603,50311780,.,-,.,ENSMUSG00000020368.14,protein_coding,...,KNOWN,Canx-201,ENSMUSP00000137440.1,"basic,appris_principal_1,CCDS",1,CCDS24633.1,2,ENSMUSE00001016144.1,NaN,"[1, 2, 1]"
4,chr11,ENSEMBL,CDS,50311603,50311777,.,-,0,ENSMUSG00000020368.14,protein_coding,...,KNOWN,Canx-201,ENSMUSP00000137440.1,"basic,appris_principal_1,CCDS",1,CCDS24633.1,2,ENSMUSE00001016144.1,NaN,"[1, 2, 1]"
5,chr11,ENSEMBL,start_codon,50311774,50311777,.,-,0,ENSMUSG00000020368.14,protein_coding,...,KNOWN,Canx-201,ENSMUSP00000137440.1,"basic,appris_principal_1,CCDS",1,CCDS24633.1,2,ENSMUSE00001016144.1,NaN,"[1, 2, 1]"
6,chr11,ENSEMBL,exon,50310793,50310867,.,-,.,ENSMUSG00000020368.14,protein_coding,...,KNOWN,Canx-201,ENSMUSP00000137440.1,"basic,appris_principal_1,CCDS",1,CCDS24633.1,3,ENSMUSE00000103860.1,NaN,"[1, 2, 1]"
7,chr11,ENSEMBL,CDS,50310793,50310867,.,-,0,ENSMUSG00000020368.14,protein_coding,...,KNOWN,Canx-201,ENSMUSP00000137440.1,"basic,appris_principal_1,CCDS",1,CCDS24633.1,3,ENSMUSE00000103860.1,NaN,"[1, 2, 1]"


,ic,gene_id,transcript_id,basic_set,appris_principal
0,50297363-50298319,ENSMUSG00000020368.14,ENSMUST00000153068.1,NaN,NaN
1,50298116-50297273-50297057-50296198-50299378-5...,ENSMUSG00000020368.14,ENSMUST00000146979.1,NaN,NaN
2,50298837-50300990,ENSMUSG00000020368.14,"ENSMUST00000155801.1,ENSMUST00000155801.2",True,1
3,50311603-50310793-50310371-50308787-50308309-5...,ENSMUSG00000020368.14,ENSMUST00000179865.6,True,1
4,50311603-50310793-50310371-50308787-50308309-5...,ENSMUSG00000020368.14,ENSMUST00000020637.8,True,1


In [203]:
df.head()

,ic,gene_id,transcript_id,basic_set,appris_principal,ic_num
0,50297363-50298319,ENSMUSG00000020368.14,ENSMUST00000153068.1,NaN,NaN,4
1,50298116-50297273-50297057-50296198-50299378-5...,ENSMUSG00000020368.14,ENSMUST00000146979.1,NaN,NaN,5
2,50298837-50300990,ENSMUSG00000020368.14,"ENSMUST00000155801.1,ENSMUST00000155801.2",True,1,1
3,50311603-50310793-50310371-50308787-50308309-5...,ENSMUSG00000020368.14,ENSMUST00000179865.6,True,1,2
4,50311603-50310793-50310371-50308787-50308309-5...,ENSMUSG00000020368.14,ENSMUST00000020637.8,True,1,3
